In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform

KFP SDK version: 2.7.0
google-cloud-aiplatform==1.44.0


In [2]:
! pip3 freeze | grep 

Usage: grep [OPTION]... PATTERNS [FILE]...
Try 'grep --help' for more information.
ERROR: Exception:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/commands/freeze.py", line 107, in run
    sys.stdout.write(line + "\n")
BrokenPipeError: [Errno 32] Broken pipe


In [3]:
import google.cloud.aiplatform as aiplatform
import kfp
from kfp import compiler, dsl
from kfp.dsl import Artifact, Dataset, Input, Metrics, Model, Output, component

In [4]:
from kfp.dsl import ClassificationMetrics
from typing import NamedTuple



In [5]:
PROJECT_ID = 'kubeflow-mlops-410520' # replace with project ID
REGION = 'us-central1'
EXPERIMENT = 'vertex-pipelines'
SERIES = 'dev'

# gcs bucket
GCS_BUCKET = PROJECT_ID
BUCKET_URI = f"gs://{PROJECT_ID}-bucket"  # @param {type:"string"}

In [6]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [6]:
@component(
    packages_to_install = [
        "pandas==1.3.4",
        "scikit-learn==1.0.1",
    ],
)
def get_data(
    dataset_train: Output[Dataset],
    dataset_test: Output[Dataset],
):

    from sklearn import datasets
    from sklearn.model_selection import train_test_split as tts
    import pandas as pd


    # dataset https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
    data_raw = datasets.load_breast_cancer()
    data = pd.DataFrame(data_raw.data, columns=data_raw.feature_names)
    data["target"] = data_raw.target

    train, test = tts(data, test_size=0.3)

    train.to_csv(dataset_train.path)
    test.to_csv(dataset_test.path)

/home/jupyter/.local/lib/python3.10/site-packages/kfp/dsl/component_decorator.py:119: FutureWarning: Python 3.7 has reached end-of-life. The default base_image used by the @dsl.component decorator will switch from 'python:3.7' to 'python:3.8' on April 23, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.8.
  return component_factory.create_component_from_func(


In [ ]:
print('hello')

In [85]:
@component(
    packages_to_install = [
        
        "xgboost==1.6.2",
        "pandas==1.3.5",
        "joblib==1.1.0",
        "scikit-learn==1.0.2",
        "pickle5==0.0.12",
        "joblib==1.1.0",
    ],
)
def train_model(
    dataset: Input[Dataset],
    model_artifact: Output[Model]
):

    from xgboost import XGBClassifier
    import pandas as pd
    import pickle5 as pickle
    import os
    #import datetime
    import subprocess
    from pathlib import Path
    import joblib
    import logging
    logging.basicConfig(level=logging.DEBUG)
    
    
    data = pd.read_csv(dataset.path)

    model = XGBClassifier(
        objective="binary:logistic"
    )
    model.fit(
        data.drop(columns=["target"]),
        data.target,
    )

    score = model.score(
        data.drop(columns=["target"]),
        data.target,
    )

    model_artifact.metadata["train_score"] = float(score)
    model_artifact.metadata["framework"] = "XGBoost"
    os.makedirs(model_artifact.path, exist_ok=True)
    joblib.dump(model, os.path.join(model_artifact.path, "model.joblib"))
    print(model_artifact.path)
    #model_filename = 'xgb_model.pkl'

    #model.save_model(model_artifact.path + "/xgb_model.pkl")
    
    #model_filename = 'xgb_model.pkl'
    # MODEL_DIR = model_artifact.path #gcs_path_to_local_path(os.environ["AIP_MODEL_DIR"])
    # MODEL_PATH = MODEL_DIR + "/model.bst"
    # print("saving model to " + MODEL_PATH) 
    # if not Path(MODEL_DIR).exists():
    #     Path(MODEL_DIR).mkdir(parents=True, exist_ok=True)
    # joblib.dump(model, MODEL_PATH) 
    # model.save_model(MODEL_PATH)
#     pickle.dump(model, open(model_filename, "wb"))
        
#     EXPORT_PATH = os.path.join(
#         model_artifact.path, datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
    
#     gcs_model_path = '{}/{}'.format(EXPORT_PATH, model_filename)
#     subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path])
#     print('Saved model in: {}'.format(gcs_model_path))  

In [4]:
print('hello')

hello


In [5]:
# from google.cloud import storage
# import joblib
# client = storage.Client()
# bucket = client.get_bucket("kubeflow-mlops-410520-bucket")
# blob = bucket.blob("gs://kubeflow-mlops-410520-bucket/pipeline_root/xgboost-pipeline/990864364836/xgboost-pipeline-with-deployment-20240327150548/train-model_4480642443057823744/model_artifact/model.bst")
# with blob.open(mode="rb") as file:
#     model = joblib.load(file)

In [76]:
# @component(
#     packages_to_install = [
#         "pandas==1.3.4",
#         "scikit-learn==1.0.1",
#         "xgboost==1.5.1",
#         "google-cloud-storage==2.13.0",
#         "google-cloud-aiplatform==1.3.0"
#     ],
# )
# def eval_model(
#     test_set: Input[Dataset],
#     xgb_model: Input[Model],
#     metrics: Output[ClassificationMetrics],
#     smetrics: Output[Metrics],
#     project : str,
#     region : str,) -> NamedTuple("Outputs", [("deploy", str)]):
#     from xgboost import XGBClassifier
#     import pandas as pd
#     import joblib
#     import logging
    
#     aiplatform.init(project=project, location=region)
#     logging.basicConfig(level=logging.DEBUG)
#     from google.cloud import storage

#     logging.debug("MODEL URI " + xgb_model.uri)
#     logging.debug("MODEL PATH " + xgb_model.path)
    
#     model_path = xgb_model.uri
#     bucket= model_path.split("//")[1] # remove gs://
#     bucket = bucket.split("/")[0]
#     path = model_path.replace("gs://"+bucket+"/", "")
    
#     logging.debug("CLOUD STORAGE " + bucket)
#     logging.debug("MODEL PATH " + path)   
    
#     client = storage.Client()
#     bucket = client.get_bucket(bucket)
#     blob = bucket.blob(path + "/model.bst")
#     with blob.open(mode="rb") as file:
#         model = joblib.load(file)
        
        
#     data = pd.read_csv(test_set.path)
#    # model = XGBClassifier()
    
    
#     #model.load_model(xgb_model.path)
#     # model = joblib.load(xgb_model.path)
#     #model = joblib.load(xgb_model.uri)

#     score = model.score(
#         data.drop(columns=["target"]),
#         data.target,
#     )

#     from sklearn.metrics import roc_curve
#     y_scores =  model.predict_proba(data.drop(columns=["target"]))[:, 1]
#     fpr, tpr, thresholds = roc_curve(
#          y_true=data.target.to_numpy(), y_score=y_scores, pos_label=True
#     )
#     metrics.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())

#     from sklearn.metrics import confusion_matrix
#     y_pred = model.predict(data.drop(columns=["target"]))

#     metrics.log_confusion_matrix(
#        ["False", "True"],
#        confusion_matrix(
#            data.target, y_pred
#        ).tolist()
#     )

#     xgb_model.metadata["test_score"] = float(score)
#     smetrics.log_metric("score", float(score))


#     deploy = "true"
#     #compare threshold or to previous

#     return (deploy,)

@component(
    packages_to_install = [
        
        "xgboost==1.6.2",
        "pandas==1.3.5",
        "joblib==1.1.0",
        "scikit-learn==1.0.2",
        "google-cloud-storage==2.13.0",
    ],
)
def eval_model(
    test_set: Input[Dataset],
    xgb_model: Input[Model],
    metrics: Output[ClassificationMetrics],
    smetrics: Output[Metrics],
    bucket_name: str,
) -> NamedTuple("Outputs", [("deploy", str)]):
    from xgboost import XGBClassifier
    import pandas as pd

    data = pd.read_csv(test_set.path)
    model = XGBClassifier()
    
    from google.cloud import storage
    import joblib
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob_path = xgb_model.uri.replace("gs://" + bucket_name + "/", "")
    smetrics.log_metric("blob_path", str(blob_path))

    blob = bucket.blob(blob_path + "/model.joblib")
    
    
    
    with blob.open(mode="rb") as file:
        model = joblib.load(file)
    #model.load_model(xgb_model.path)
    

    score = model.score(
        data.drop(columns=["target"]),
        data.target,
    )

    from sklearn.metrics import roc_curve
    y_scores =  model.predict_proba(data.drop(columns=["target"]))[:, 1]
    fpr, tpr, thresholds = roc_curve(
         y_true=data.target.to_numpy(), y_score=y_scores, pos_label=True
    )
    metrics.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())

    from sklearn.metrics import confusion_matrix
    y_pred = model.predict(data.drop(columns=["target"]))

    metrics.log_confusion_matrix(
       ["False", "True"],
       confusion_matrix(
           data.target, y_pred
       ).tolist()
    )

    xgb_model.metadata["test_score"] = float(score)
    smetrics.log_metric("score", float(score))


    deploy = "true"
    #compare threshold or to previous

    return (deploy,)

In [77]:
@component(
    packages_to_install=["google-cloud-aiplatform==1.25.0"],
)
def deploy_xgboost_model(
    model: Input[Model],
    project_id: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model],
):
    """Deploys an XGBoost model to Vertex AI Endpoint.

    Args:
        model: The model to deploy.
        project_id: The project ID of the Vertex AI Endpoint.

    Returns:
        vertex_endpoint: The deployed Vertex AI Endpoint.
        vertex_model: The deployed Vertex AI Model.
    """
    from google.cloud import aiplatform

    aiplatform.init(project=project_id)

    deployed_model = aiplatform.Model.upload(
        display_name="breast-cancer-xgb-classification",
        artifact_uri=model.uri,
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-6:latest",
    )
    endpoint = deployed_model.deploy(machine_type="n1-standard-4")

    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

In [86]:
BUCKET_NAME="gs://" + PROJECT_ID + "-bucket"
PIPELINE_ROOT = BUCKET_NAME + "/pipeline_root/"

In [87]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT + "xgboost-pipeline-v2",
    # A name for the pipeline. Use to determine the pipeline Context.
    name="xgboost-pipeline-with-deployment-v2",
)
def pipeline():
    dataset_op = get_data()
    training_op = train_model(dataset = dataset_op.outputs["dataset_train"])
    eval_op = eval_model(
        test_set=dataset_op.outputs["dataset_test"],
        xgb_model=training_op.outputs["model_artifact"],
        bucket_name = "kubeflow-mlops-410520-bucket"
    )

    with dsl.Condition(
        eval_op.outputs["deploy"] == "true",
        name="deploy",
    ):

      deploy_op = deploy_xgboost_model(model = training_op.outputs["model_artifact"],
                         project_id = PROJECT_ID,
                        )


/var/tmp/ipykernel_32293/1289652284.py:16: DeprecationWarning: dsl.Condition is deprecated. Please use dsl.If instead.
  with dsl.Condition(


In [88]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline-breast-cancer.yaml")

In [89]:
from google.cloud.aiplatform import pipeline_jobs


In [82]:
# job = pipeline_jobs.PipelineJob(
#     display_name="xgboost test",
#     template_path="xgb_pipeline.json",
#     enable_caching = False
# )

In [90]:
job = aiplatform.PipelineJob(
    display_name="breast-cancer-demo-pipeline",
    template_path="pipeline-breast-cancer.yaml",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/990864364836/locations/us-central1/pipelineJobs/xgboost-pipeline-with-deployment-v2-20240328202049
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/990864364836/locations/us-central1/pipelineJobs/xgboost-pipeline-with-deployment-v2-20240328202049')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/xgboost-pipeline-with-deployment-v2-20240328202049?project=990864364836
PipelineJob projects/990864364836/locations/us-central1/pipelineJobs/xgboost-pipeline-with-deployment-v2-20240328202049 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/990864364836/locations/us-central1/pipelineJobs/xgboost-pipeline-with-deployment-v2-20240328202049 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/990864364836/locations/us-central1/pipelineJobs/xgboost-pipeline-with-deployment-v2-20240328202049 cur

RetryError: Timeout of 120.0s exceeded, last exception: 503 502:Bad Gateway